In [19]:
import pandas as pd
import numpy as np

import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import matplotlib

import geopandas
import pandas as pd
import numpy as np
import folium
import gc
from folium.plugins import TimeSliderChoropleth
from scipy.signal import find_peaks

import warnings
warnings.filterwarnings('ignore')

In [20]:
df_bacterias = pd.read_csv('datos/dataset_procesado_bacterias_resistentes.csv')
df_bacterias.head()

,Unnamed: 0,Sample_ID,Year,Country,Continent,Beta.lactamase,Azithromycin,Ciprofloxacin,Ceftriaxone,Cefixime,Tetracycline,Penicillin,NG_MAST,Group,azm_mic,cip_mic,cro_mic,cfx_mic,tet_mic,pen_mic,log2_azm_mic,log2_cip_mic,log2_cro_mic,log2_cfx_mic,log2_tet_mic,log2_pen_mic,azm_sr,cip_sr,cro_sr,cfx_sr,tet_sr,pen_sr
0,0,ERR1549286,2015.0,UK,Europe,NaN,>256,NaN,0.016,NaN,NaN,NaN,9768,1058,512.0,NaN,0.016,NaN,NaN,NaN,9.0,NaN,-5.965784,NaN,NaN,NaN,1.0,NaN,0.0,NaN,NaN,NaN
1,1,ERR1549290,2015.0,UK,Europe,NaN,>256,NaN,0.004,NaN,NaN,NaN,9768,1058,512.0,NaN,0.004,NaN,NaN,NaN,9.0,NaN,-7.965784,NaN,NaN,NaN,1.0,NaN,0.0,NaN,NaN,NaN
2,2,ERR1549291,2015.0,UK,Europe,NaN,>256,NaN,0.006,NaN,NaN,NaN,9768,1058,512.0,NaN,0.006,NaN,NaN,NaN,9.0,NaN,-7.380822,NaN,NaN,NaN,1.0,NaN,0.0,NaN,NaN,NaN
3,3,ERR1549287,2015.0,UK,Europe,NaN,>256,NaN,0.006,NaN,NaN,NaN,9768,1058,512.0,NaN,0.006,NaN,NaN,NaN,9.0,NaN,-7.380822,NaN,NaN,NaN,1.0,NaN,0.0,NaN,NaN,NaN
4,4,ERR1549288,2015.0,UK,Europe,NaN,>256,NaN,0.008,NaN,NaN,NaN,9768,1058,512.0,NaN,0.008,NaN,NaN,NaN,9.0,NaN,-6.965784,NaN,NaN,NaN,1.0,NaN,0.0,NaN,NaN,NaN


In [21]:
df_bacterias.isna().any()

Unnamed: 0        False
Sample_ID         False
Year               True
Country            True
Continent          True
Beta.lactamase     True
Azithromycin       True
Ciprofloxacin      True
Ceftriaxone        True
Cefixime           True
Tetracycline       True
Penicillin         True
NG_MAST            True
Group             False
azm_mic            True
                  ...  
cfx_mic            True
tet_mic            True
pen_mic            True
log2_azm_mic       True
log2_cip_mic       True
log2_cro_mic       True
log2_cfx_mic       True
log2_tet_mic       True
log2_pen_mic       True
azm_sr             True
cip_sr             True
cro_sr             True
cfx_sr             True
tet_sr             True
pen_sr             True
Length: 32, dtype: bool

In [22]:
df_bacterias.dropna(subset=['Year'], inplace=True)

In [23]:
def get_hex_colors(df, data_to_color, cmap = matplotlib.cm.Reds, log = False):
    
    '''
    This function takes the following arguments
        1. df:pandas DataFrame with the data.
        2. data_to_color: the column name with data based on which we want to create the color scale.
        3. cmap: colors you want to plot. You can use this to communicate different messages. For example: greens --> good, greys --> deaths.
                default is matplotlib.cm.Reds
                more about colormaps: https://matplotlib.org/3.1.1/gallery/color/colormap_reference.html
        3. log: if data has huge outliers, we can create the color map with a logarithic normalization. This way, the outliers won't "pale" our other data.
                default is False.
        
    '''
    
    cmap = cmap # define the color pallete you want. You can use Reds, Blues, Greens etc
    my_values = df[data_to_color] # get the value you wan to convert to colors
    
    mini = min(my_values) # get the min to normalize
    maxi= max(my_values) # get the max to normalize
    
    LOGMIN = 0.01 # arbitrary lower bound for log scale
    
    if log: 
        norm = matplotlib.colors.LogNorm(vmin=max(mini,LOGMIN), vmax=maxi) # normalize log data
    else:
        norm = matplotlib.colors.Normalize(vmin=mini, vmax=maxi) # create a color range
        
    colors = {value:matplotlib.colors.rgb2hex(cmap(norm(value))[:3]) for value in sorted(list(set(my_values)))} # create a dictionary with the total_infected or deaths as keys and colors as values
    
    return colors

In [24]:
def get_hex_colors_2(value, cats):
    '''
    Color paletter used from this website:
    
    https://colorbrewer2.org/#type=sequential&scheme=Reds&n=9
    
    The color selection will be based on the percentile each value is in.
    '''
    if value == 0:
        return "#FFFFFF"
    elif value in cats[0]:
        return "#fff5f0"
    elif value in cats[1]:
        return "#fee0d2"
    elif value in cats[2]:
        return "#fcbba1"
    elif value in cats[3]:
        return "#fc9272"
    elif value in cats[4]:
        return "#fb6a4a"
    elif value in cats[5]:
        return "#ef3b2c"
    elif value in cats[6]:
        return "#cb181d"
    elif value in cats[7]:
        return "#a50f15"
    elif value in cats[8]:
        return "#67000d"
    else:
        return "#000000"

In [25]:
def correct_date(date_str):
    list_dates = date_str.split("-")
    year = list_dates[0]
    month = list_dates[1]
    day = list_dates[2]
    
    if len(day) == 1:
        day = "0" + day
    if len(month) == 1:
        month = "0" + month
        
    return "/".join([day, month, year])

In [26]:
def correct_date2(year_float):
    if(float('nan') != year_float):
        year = str(year_float)[:4]
    else:
        year='2000'
    day = '01'
    month = '01'
    
    return "/".join([day, month, year])

In [27]:
pd.set_option('display.max_rows', 0)
pd.set_option('display.max_columns', 0)
PATH_GEO_JSON = 'datos/ne_110m_admin_0_countries.json'
gdf = geopandas.read_file(PATH_GEO_JSON)
gdf.head()

,_isDeleted,featurecla,scalerank,LABELRANK,SOVEREIGNT,SOV_A3,ADM0_DIF,LEVEL,TYPE,TLC,ADMIN,ADM0_A3,GEOU_DIF,GEOUNIT,GU_A3,SU_DIF,SUBUNIT,SU_A3,BRK_DIFF,NAME,NAME_LONG,BRK_A3,BRK_NAME,BRK_GROUP,ABBREV,POSTAL,FORMAL_EN,FORMAL_FR,NAME_CIAWF,NOTE_ADM0,NOTE_BRK,NAME_SORT,NAME_ALT,MAPCOLOR7,MAPCOLOR8,MAPCOLOR9,MAPCOLOR13,POP_EST,POP_RANK,POP_YEAR,GDP_MD,GDP_YEAR,ECONOMY,INCOME_GRP,FIPS_10,ISO_A2,ISO_A2_EH,ISO_A3,ISO_A3_EH,ISO_N3,ISO_N3_EH,UN_A3,WB_A2,WB_A3,WOE_ID,WOE_ID_EH,WOE_NOTE,ADM0_ISO,ADM0_DIFF,ADM0_TLC,...,NAME_BN,NAME_DE,NAME_EN,NAME_ES,NAME_FA,NAME_FR,NAME_EL,NAME_HE,NAME_HI,NAME_HU,NAME_ID,NAME_IT,NAME_JA,NAME_KO,NAME_NL,NAME_PL,NAME_PT,NAME_RU,NAME_SV,NAME_TR,NAME_UK,NAME_UR,NAME_VI,NAME_ZH,NAME_ZHT,FCLASS_ISO,TLC_DIFF,FCLASS_TLC,FCLASS_US,FCLASS_FR,FCLASS_RU,FCLASS_ES,FCLASS_CN,FCLASS_TW,FCLASS_IN,FCLASS_NP,FCLASS_PK,FCLASS_DE,FCLASS_GB,FCLASS_BR,FCLASS_IL,FCLASS_PS,FCLASS_SA,FCLASS_EG,FCLASS_MA,FCLASS_PT,FCLASS_AR,FCLASS_JP,FCLASS_KO,FCLASS_VN,FCLASS_TR,FCLASS_ID,FCLASS_PL,FCLASS_GR,FCLASS_IT,FCLASS_NL,FCLASS_SE,FCLASS_BD,FCLASS_UA,geometry
0,False,Admin-0 country,1,6,Fiji,FJI,0,2,Sovereign country,1,Fiji,FJI,0,Fiji,FJI,0,Fiji,FJI,0,Fiji,Fiji,FJI,Fiji,,Fiji,FJ,Republic of Fiji,,Fiji,,,Fiji,,5,1,2,2,889953.0,11,2019,5496,2019,6. Developing region,4. Lower middle income,FJ,FJ,FJ,FJI,FJI,242,242,242,FJ,FJI,23424813,23424813,Exact WOE match as country,FJI,,FJI,...,à¦«à¦¿à¦à¦¿,Fidschi,Fiji,Fiyi,ÙÛØ¬Û,Fidji,Î¦Î¯ÏÎ¶Î¹,×¤××'×,à¤«à¤¼à¤¿à¤à¥,Fidzsi-szigetek,Fiji,Figi,ãã£ã¸ã¼,í¼ì§,Fiji,FidÅ¼i,Fiji,Ð¤Ð¸Ð´Ð¶Ð¸,Fiji,Fiji,Ð¤ÑÐ´Ð¶Ñ,ÙØ¬Û,Fiji,ææµ,ææ¿,Admin-0 country,,Admin-0 country,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,"POLYGON ((180.00000 -16.06712, 180.00000 -16.5..."
1,False,Admin-0 country,1,3,United Republic of Tanzania,TZA,0,2,Sovereign country,1,United Republic of Tanzania,TZA,0,Tanzania,TZA,0,Tanzania,TZA,0,Tanzania,Tanzania,TZA,Tanzania,,Tanz.,TZ,United Republic of Tanzania,,Tanzania,,,Tanzania,,3,6,2,2,58005463.0,16,2019,63177,2019,7. Least developed region,5. Low income,TZ,TZ,TZ,TZA,TZA,834,834,834,TZ,TZA,23424973,23424973,Exact WOE match as country,TZA,,TZA,...,à¦¤à¦¾à¦¨à¦à¦¾à¦¨à¦¿à¦¯à¦¼à¦¾,Tansania,Tanzania,Tanzania,ØªØ§ÙØ²Ø§ÙÛØ§,Tanzanie,Î¤Î±Î½Î¶Î±Î½Î¯Î±,×× ×× ××,à¤¤à¤à¤à¤¼à¤¾à¤¨à¤¿à¤¯à¤¾,TanzÃ¡nia,Tanzania,Tanzania,ã¿ã³ã¶ãã¢,íìëì,Tanzania,Tanzania,TanzÃ¢nia,Ð¢Ð°Ð½Ð·Ð°Ð½Ð¸Ñ,Tanzania,Tanzanya,Ð¢Ð°Ð½Ð·Ð°Ð½ÑÑ,ØªÙØ²Ø§ÙÛÛ,Tanzania,å¦æ¡å°¼äº,å¦å°å°¼äº,Admin-0 country,,Admin-0 country,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,"POLYGON ((33.90369 -0.95000, 34.07260 -1.05982..."
2,False,Admin-0 country,1,7,Western Sahara,SAH,0,2,Indeterminate,1,Western Sahara,SAH,0,Western Sahara,SAH,0,Western Sahara,SAH,1,W. Sahara,Western Sahara,B28,W. Sahara,,W. Sah.,WS,Sahrawi Arab Democratic Republic,,Western Sahara,,Self admin.; Claimed by Morocco,Western Sahara,,4,7,4,4,603253.0,11,2017,907,2007,7. Least developed region,5. Low income,WI,EH,EH,ESH,ESH,732,732,732,-99,-99,23424990,23424990,Exact WOE match as country,B28,,B28,...,à¦ªà¦¶à§à¦à¦¿à¦® à¦¸à¦¾à¦¹à¦¾à¦°à¦¾,Westsahara,Western Sahara,Sahara Occidental,ØµØ­Ø±Ø§Û ØºØ±Ø¨Û,Sahara occidental,ÎÏ ÏÎ¹ÎºÎ® Î£Î±ÏÎ¬ÏÎ±,×¡××¨× ×××¢×¨×××ª,à¤ªà¤¶à¥à¤à¤¿à¤®à¥ à¤¸à¤¹à¤¾à¤°à¤¾,Nyugat-Szahara,Sahara Barat,Sahara Occidentale,è¥¿ãµãã©,ìì¬íë¼,Westelijke Sahara,Sahara Zachodnia,Sara Ocidental,ÐÐ°Ð¿Ð°Ð´Ð½Ð°Ñ Ð¡Ð°Ñ Ð°ÑÐ°,VÃ¤stsahara,BatÄ± Sahra,ÐÐ°Ñ ÑÐ´Ð½Ð° Ð¡Ð°Ñ Ð°ÑÐ°,Ù ØºØ±Ø¨Û ØµØ­Ø§Ø±Ø§,TÃ¢y Sahara,è¥¿æåæ,è¥¿æåæ,Admin-0 dependency,,Admin-0 dependency,,Unrecognized,,,,,Unrecognized,,,,,,,Unrecognized,Unrecognized,,Unrecognized,,,,,,Unrecognized,Unrecognized,Unrecognized,,,Unrecognized,,,,"POLYGON ((-8.66559 27.65642, -8.66512 27.58948..."
3,False,Admin-0 country,1,2,Canada,CAN,0,2,Sovereign country,1,Canada,CAN,0,Canada,CAN,0,Canada,CAN,0,Canada,Canada,CAN,Canada,,Can.,CA,Canada,,Canada,,,Canada,,6,6,2,2,37589262.0,15,2019,1736425,2019,1. Developed region: G7,1. High income: OECD,CA,CA,CA,CAN,CAN,124,124,124,CA,CAN,2

In [28]:
pd.options.display.max_rows = 0
gdf.iloc[1]
gdf.columns.values

array(['_isDeleted', 'featurecla', 'scalerank', 'LABELRANK', 'SOVEREIGNT',
       'SOV_A3', 'ADM0_DIF', 'LEVEL', 'TYPE', 'TLC', 'ADMIN', 'ADM0_A3',
       'GEOU_DIF', 'GEOUNIT', 'GU_A3', 'SU_DIF', 'SUBUNIT', 'SU_A3',
       'BRK_DIFF', 'NAME', 'NAME_LONG', 'BRK_A3', 'BRK_NAME', 'BRK_GROUP',
       'ABBREV', 'POSTAL', 'FORMAL_EN', 'FORMAL_FR', 'NAME_CIAWF',
       'NOTE_ADM0', 'NOTE_BRK', 'NAME_SORT', 'NAME_ALT', 'MAPCOLOR7',
       'MAPCOLOR8', 'MAPCOLOR9', 'MAPCOLOR13', 'POP_EST', 'POP_RANK',
       'POP_YEAR', 'GDP_MD', 'GDP_YEAR', 'ECONOMY', 'INCOME_GRP',
       'FIPS_10', 'ISO_A2', 'ISO_A2_EH', 'ISO_A3', 'ISO_A3_EH', 'ISO_N3',
       'ISO_N3_EH', 'UN_A3', 'WB_A2', 'WB_A3', 'WOE_ID', 'WOE_ID_EH',
       'WOE_NOTE', 'ADM0_ISO', 'ADM0_DIFF', 'ADM0_TLC', 'ADM0_A3_US',
       'ADM0_A3_FR', 'ADM0_A3_RU', 'ADM0_A3_ES', 'ADM0_A3_CN',
       'ADM0_A3_TW', 'ADM0_A3_IN', 'ADM0_A3_NP', 'ADM0_A3_PK',
       'ADM0_A3_DE', 'ADM0_A3_GB', 'ADM0_A3_BR', 'ADM0_A3_IL',
       'ADM0_A3_PS', 'ADM0_A3_SA

In [29]:
gdf = gdf[['FIPS_10', 'SOVEREIGNT', 'geometry']]

In [30]:
gdf.rename(columns={'FIPS_10': 'Country',
           'SOVEREIGNT': 'State',
           'geometry':'Geometry'}, inplace=True)

In [31]:
gdf.head()

,Country,State,Geometry
0,FJ,Fiji,"POLYGON ((180.00000 -16.06712, 180.00000 -16.5..."
1,TZ,United Republic of Tanzania,"POLYGON ((33.90369 -0.95000, 34.07260 -1.05982..."
2,WI,Western Sahara,"POLYGON ((-8.66559 27.65642, -8.66512 27.58948..."
3,CA,Canada,"POLYGON ((-122.83997 49.00000, -122.97418 49.0..."
4,US,United States of America,"POLYGON ((-122.83997 49.00000, -120.00000 49.0..."


In [32]:
df_bacterias_representacion = df_bacterias.merge(gdf, on='Country', how='left').drop(['Unnamed: 0'], axis=1)

In [33]:
df_bacterias_representacion.head()

,Sample_ID,Year,Country,Continent,Beta.lactamase,Azithromycin,Ciprofloxacin,Ceftriaxone,Cefixime,Tetracycline,Penicillin,NG_MAST,Group,azm_mic,cip_mic,cro_mic,cfx_mic,tet_mic,pen_mic,log2_azm_mic,log2_cip_mic,log2_cro_mic,log2_cfx_mic,log2_tet_mic,log2_pen_mic,azm_sr,cip_sr,cro_sr,cfx_sr,tet_sr,pen_sr,State,Geometry
0,ERR1549286,2015.0,UK,Europe,NaN,>256,NaN,0.016,NaN,NaN,NaN,9768,1058,512.0,NaN,0.016,NaN,NaN,NaN,9.0,NaN,-5.965784,NaN,NaN,NaN,1.0,NaN,0.0,NaN,NaN,NaN,United Kingdom,"POLYGON ((-6.19788 53.86755, -6.95373 54.07370..."
1,ERR1549290,2015.0,UK,Europe,NaN,>256,NaN,0.004,NaN,NaN,NaN,9768,1058,512.0,NaN,0.004,NaN,NaN,NaN,9.0,NaN,-7.965784,NaN,NaN,NaN,1.0,NaN,0.0,NaN,NaN,NaN,United Kingdom,"POLYGON ((-6.19788 53.86755, -6.95373 54.07370..."
2,ERR1549291,2015.0,UK,Europe,NaN,>256,NaN,0.006,NaN,NaN,NaN,9768,1058,512.0,NaN,0.006,NaN,NaN,NaN,9.0,NaN,-7.380822,NaN,NaN,NaN,1.0,NaN,0.0,NaN,NaN,NaN,United Kingdom,"POLYGON ((-6.19788 53.86755, -6.95373 54.07370..."
3,ERR1549287,2015.0,UK,Europe,NaN,>256,NaN,0.006,NaN,NaN,NaN,9768,1058,512.0,NaN,0.006,NaN,NaN,NaN,9.0,NaN,-7.380822,NaN,NaN,NaN,1.0,NaN,0.0,NaN,NaN,NaN,United Kingdom,"POLYGON ((-6.19788 53.86755, -6.95373 54.07370..."
4,ERR1549288,2015.0,UK,Europe,NaN,>256,NaN,0.008,NaN,NaN,NaN,9768,1058,512.0,NaN,0.008,NaN,NaN,NaN,9.0,NaN,-6.965784,NaN,NaN,NaN,1.0,NaN,0.0,NaN,NaN,NaN,United Kingdom,"POLYGON ((-6.19788 53.86755, -6.95373 54.07370..."


In [34]:
df_bacterias["NEW_DATE"] = df_bacterias["Year"].apply(correct_date2)
df_bacterias.drop(['Unnamed: 0'], inplace=True, axis=1)
df_bacterias["DATE"] = pd.to_datetime(df_bacterias["NEW_DATE"], format='%d/%m/%Y')
df_bacterias=df_bacterias.drop(['NEW_DATE'], axis=1)
df_bacterias=df_bacterias.drop(['Year'], axis=1)
df_bacterias.head()

,Sample_ID,Country,Continent,Beta.lactamase,Azithromycin,Ciprofloxacin,Ceftriaxone,Cefixime,Tetracycline,Penicillin,NG_MAST,Group,azm_mic,cip_mic,cro_mic,cfx_mic,tet_mic,pen_mic,log2_azm_mic,log2_cip_mic,log2_cro_mic,log2_cfx_mic,log2_tet_mic,log2_pen_mic,azm_sr,cip_sr,cro_sr,cfx_sr,tet_sr,pen_sr,DATE
0,ERR1549286,UK,Europe,NaN,>256,NaN,0.016,NaN,NaN,NaN,9768,1058,512.0,NaN,0.016,NaN,NaN,NaN,9.0,NaN,-5.965784,NaN,NaN,NaN,1.0,NaN,0.0,NaN,NaN,NaN,2015-01-01
1,ERR1549290,UK,Europe,NaN,>256,NaN,0.004,NaN,NaN,NaN,9768,1058,512.0,NaN,0.004,NaN,NaN,NaN,9.0,NaN,-7.965784,NaN,NaN,NaN,1.0,NaN,0.0,NaN,NaN,NaN,2015-01-01
2,ERR1549291,UK,Europe,NaN,>256,NaN,0.006,NaN,NaN,NaN,9768,1058,512.0,NaN,0.006,NaN,NaN,NaN,9.0,NaN,-7.380822,NaN,NaN,NaN,1.0,NaN,0.0,NaN,NaN,NaN,2015-01-01
3,ERR1549287,UK,Europe,NaN,>256,NaN,0.006,NaN,NaN,NaN,9768,1058,512.0,NaN,0.006,NaN,NaN,NaN,9.0,NaN,-7.380822,NaN,NaN,NaN,1.0,NaN,0.0,NaN,NaN,NaN,2015-01-01
4,ERR1549288,UK,Europe,NaN,>256,NaN,0.008,NaN,NaN,NaN,9768,1058,512.0,NaN,0.008,NaN,NaN,NaN,9.0,NaN,-6.965784,NaN,NaN,NaN,1.0,NaN,0.0,NaN,NaN,NaN,2015-01-01


In [35]:
df_bacterias["DATE_for_Folium"] = (df_bacterias["DATE"].astype('int64')// 10**9).astype('U10')
df_bacterias.drop(['DATE'], axis=1, inplace=True)
df_bacterias['Date'] = df_bacterias['DATE_for_Folium']
df_bacterias.drop(['DATE_for_Folium'], axis=1, inplace=True)
df_bacterias.head()

,Sample_ID,Country,Continent,Beta.lactamase,Azithromycin,Ciprofloxacin,Ceftriaxone,Cefixime,Tetracycline,Penicillin,NG_MAST,Group,azm_mic,cip_mic,cro_mic,cfx_mic,tet_mic,pen_mic,log2_azm_mic,log2_cip_mic,log2_cro_mic,log2_cfx_mic,log2_tet_mic,log2_pen_mic,azm_sr,cip_sr,cro_sr,cfx_sr,tet_sr,pen_sr,Date
0,ERR1549286,UK,Europe,NaN,>256,NaN,0.016,NaN,NaN,NaN,9768,1058,512.0,NaN,0.016,NaN,NaN,NaN,9.0,NaN,-5.965784,NaN,NaN,NaN,1.0,NaN,0.0,NaN,NaN,NaN,1420070400
1,ERR1549290,UK,Europe,NaN,>256,NaN,0.004,NaN,NaN,NaN,9768,1058,512.0,NaN,0.004,NaN,NaN,NaN,9.0,NaN,-7.965784,NaN,NaN,NaN,1.0,NaN,0.0,NaN,NaN,NaN,1420070400
2,ERR1549291,UK,Europe,NaN,>256,NaN,0.006,NaN,NaN,NaN,9768,1058,512.0,NaN,0.006,NaN,NaN,NaN,9.0,NaN,-7.380822,NaN,NaN,NaN,1.0,NaN,0.0,NaN,NaN,NaN,1420070400
3,ERR1549287,UK,Europe,NaN,>256,NaN,0.006,NaN,NaN,NaN,9768,1058,512.0,NaN,0.006,NaN,NaN,NaN,9.0,NaN,-7.380822,NaN,NaN,NaN,1.0,NaN,0.0,NaN,NaN,NaN,1420070400
4,ERR1549288,UK,Europe,NaN,>256,NaN,0.008,NaN,NaN,NaN,9768,1058,512.0,NaN,0.008,NaN,NaN,NaN,9.0,NaN,-6.965784,NaN,NaN,NaN,1.0,NaN,0.0,NaN,NaN,NaN,1420070400


In [36]:
m = folium.Map(location = (40, 0), zoom_start = 5.5)

folium.Choropleth(
    geo_data = df_bacterias,
    name = 'choropleth',
    data = df_bacterias[df_bacterias["Date"] == max(df_bacterias["Date"])],
    columns = ['Country', 'NG_MAST'],
    key_on='feature.properties.id',
    fill_color='RdPu',
    fill_opacity=0.7,
    line_opacity=0.2,
    legend_name = 'Mapa segun capacidad de cura'
).add_to(m)

m

TypeError: ufunc 'isnan' not supported for the input types, and the inputs could not be safely coerced to any supported types according to the casting rule ''safe''